In [1]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from urllib.request import URLError
from copy import deepcopy
import numpy as np
import pickle
import time as tm
import undetected_chromedriver as uc



In [2]:
perfume_links = []

In [47]:
driver = uc.Chrome()
gender = "unisex"
driver.get("https://www.fragrantica.com/search/?godina={}%3A{}&spol={}".format(2023, "", gender))

In [48]:
link_elements = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[1]/div[1]/div/div/div/div[2]/div[1]/div/div[3]/div/div/div/span/div').find_elements(By.XPATH, '//div[2]/p[1]/a')
for element in link_elements:
    perfume_links.append(element.get_attribute('href'))

In [49]:
with open("frag_links.pkl", "wb") as f:
    pickle.dump(perfume_links, f)

In [107]:
with open("frag_links.pkl", "rb") as f:
    perfume_links = pickle.load(f)

In [ ]:
await bot.send_message(chat_id=CHAT_ID, text="Scraping has begun.")

last_update_id = 905198177

while count < len(perfume_links):
    try:
        latest_update_id = (await bot.get_updates())[-1].update_id
        if latest_update_id != last_update_id:
            last_update_id = latest_update_id
            await bot.send_message(chat_id=CHAT_ID, text="Count: {}".format(count))
        print(count)
        link = perfume_links[count]
        new_driver = uc.Chrome()
        new_driver.get(link)

        tm.sleep(25)
        new_driver.execute_script("window.stop();")
        
        soup = BeautifulSoup(new_driver.page_source, "html.parser")
        brand, perfume = link.split('/')[-2:]

        with open("cached_frags_soups/{}-{}.pkl".format(brand, perfume), "wb") as f:
            pickle.dump(soup, f)

        count += 1
        tm.sleep(6)
    except URLError as e:
        print("URLError: " + str(e))
        print("Retrying...")
    except TimeoutException as e:
        print("TimeoutException: " + str(e))
        print("Retrying...")
    except ValueError as e:
        print("ValueError: " + str(e))
        print("Retrying...")
    except NoSuchElementException as e:
        print("NoSuchElementException: " + str(e))
        if new_driver.current_url == 'https://www.fragrantica.com/':
            count += 1
            print("Moving on to next perfume...")
        else:
            print("Retrying...")
    except Exception as e:
        await bot.send_message(chat_id=CHAT_ID, text="An error has occured.")
        raise
    finally:
        new_driver.execute_script("window.stop();")
        new_driver.close()
        new_driver.quit()
        
await bot.send_message(chat_id=CHAT_ID, text="Scraping has ended.")

In [100]:
# while count < len(perfume_links):
#     try:
#         print(count)
#         link = perfume_links[count]
#         new_driver = uc.Chrome()
#         new_driver.get(link)

#         tm.sleep(25)
#         new_driver.execute_script("window.stop();")

#         name = new_driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[4]/div[1]/div[1]/div/div[1]/h1').text
#         company = new_driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[4]/div[1]/div[1]/div/div[2]/div[1]/div[2]/p/a/span').text
#         accords = {}

#         for element in new_driver.find_element(By.XPATH, '//*[@id="main-content"]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div').find_elements(By.XPATH, ".//div[*]"):
#             accords[element.text] = float(element.find_element(By.XPATH, ".//div").get_attribute('style').split(';')[-2].split(':')[-1][1:-1])

#         likability_season_time_raw_elements = new_driver.find_elements(By.CLASS_NAME, 'voting-small-chart-size')[3:]

#         likability = {}
#         likability['love'] = float(likability_season_time_raw_elements[0].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         likability['like'] = float(likability_season_time_raw_elements[1].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         likability['ok'] = float(likability_season_time_raw_elements[2].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         likability['dislike'] = float(likability_season_time_raw_elements[3].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         likability['hate'] = float(likability_season_time_raw_elements[4].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])

#         season = {}
#         season['winter'] = float(likability_season_time_raw_elements[5].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         season['spring'] = float(likability_season_time_raw_elements[6].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         season['summer'] = float(likability_season_time_raw_elements[7].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         season['fall'] = float(likability_season_time_raw_elements[8].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])

#         time = {}
#         time['day'] = float(likability_season_time_raw_elements[9].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])
#         time['night'] = float(likability_season_time_raw_elements[10].find_element(By.XPATH, './/div/div').get_attribute('style').split(';')[-3].split(':')[-1][1:-1])

#         year = None
#         description = new_driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[4]/div[1]/div[1]/div/div[2]/div[5]/div/p[1]').text
#         launched_in_substr_idx = description.find('was launched in ')
#         if launched_in_substr_idx != -1:
#             year = int(description[launched_in_substr_idx+16:launched_in_substr_idx+20])

#         rating = float(new_driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[4]/div[1]/div[1]/div/div[2]/div[4]/div[3]/div/p[1]/span[1]').text)

#         num_reviews = 0
#         num_reviews_str = new_driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[4]/div[1]/div[1]/div/div[2]/div[4]/div[3]/div/p[1]/span[3]').text
#         num_reviews_split = num_reviews_str.split(',')
#         num_reviews += int(num_reviews_split[-1])
#         if len(num_reviews_split) > 1:
#             num_reviews += 1000*int(num_reviews_split[-2])

#         notes = {}
#         try:
#             notes_parent_elements = new_driver.find_element(By.ID, 'pyramid').find_element(By.XPATH, ".//div/div/div[2]").find_elements(By.XPATH, './div')
#             notes_type_elements = new_driver.find_element(By.ID, 'pyramid').find_element(By.XPATH, ".//div/div/div[2]").find_elements(By.XPATH, './h4/b')
#             if len(notes_type_elements) == 0:
#                 for note_element in notes_parent_elements[2].find_elements(By.XPATH, './div/div'):
#                     notes[note_element.find_element(By.XPATH, "./div[2]").text] = float(note_element.find_element(By.XPATH, "./div[1]/img").get_attribute('style')[7:-4])
#             else:
#                 for i in range(2, len(notes_parent_elements)):
#                     notes[notes_type_elements[i-2].text] = {}
#                     for note_element in notes_parent_elements[i].find_elements(By.XPATH, './div/div'):
#                         notes[notes_type_elements[i-2].text][note_element.find_element(By.XPATH, "./div[2]").text] = float(note_element.find_element(By.XPATH, "./div[1]/img").get_attribute('style')[7:-4])
#         except NoSuchElementException:
#             pass

#         longevity_sillage_gender_price_value_elements = new_driver.find_elements(By.CLASS_NAME, 'alert')

#         longevity = {}
#         longevity['very_weak'] = int(longevity_sillage_gender_price_value_elements[0].get_attribute('value'))
#         longevity['weak'] = int(longevity_sillage_gender_price_value_elements[1].get_attribute('value'))
#         longevity['moderate'] = int(longevity_sillage_gender_price_value_elements[2].get_attribute('value'))
#         longevity['long_lasting'] = int(longevity_sillage_gender_price_value_elements[3].get_attribute('value'))
#         longevity['eternal'] = int(longevity_sillage_gender_price_value_elements[4].get_attribute('value'))

#         sillage = {}
#         sillage['intimate'] = int(longevity_sillage_gender_price_value_elements[5].get_attribute('value'))
#         sillage['moderate'] = int(longevity_sillage_gender_price_value_elements[6].get_attribute('value'))
#         sillage['strong'] = int(longevity_sillage_gender_price_value_elements[7].get_attribute('value'))
#         sillage['enormous'] = int(longevity_sillage_gender_price_value_elements[8].get_attribute('value'))

#         gender = {}
#         gender['female'] = int(longevity_sillage_gender_price_value_elements[9].get_attribute('value'))
#         gender['more_female'] = int(longevity_sillage_gender_price_value_elements[10].get_attribute('value'))
#         gender['unisex'] = int(longevity_sillage_gender_price_value_elements[11].get_attribute('value'))
#         gender['more_male'] = int(longevity_sillage_gender_price_value_elements[12].get_attribute('value'))
#         gender['male'] = int(longevity_sillage_gender_price_value_elements[13].get_attribute('value'))

#         price_value = {}
#         price_value['way_overpriced'] = int(longevity_sillage_gender_price_value_elements[14].get_attribute('value'))
#         price_value['overpriced'] = int(longevity_sillage_gender_price_value_elements[15].get_attribute('value'))
#         price_value['ok'] = int(longevity_sillage_gender_price_value_elements[16].get_attribute('value'))
#         price_value['good_value'] = int(longevity_sillage_gender_price_value_elements[17].get_attribute('value'))
#         price_value['great_value'] = int(longevity_sillage_gender_price_value_elements[18].get_attribute('value'))

#         similar_perfumes = {}
#         similar_perfumes_elements = new_driver.find_elements(By.CLASS_NAME, 'flickity-slider')[-2].find_elements(By.XPATH, "./div")
#         next_button = new_driver.find_elements(By.CLASS_NAME, "flickity-button")[-3]
#         new_driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
#         hit_count = -1
#         for element in similar_perfumes_elements:
#             upvotes = int(element.find_element(By.XPATH, "./div/div/div/div[1]/div/span").text)
#             hit_count += 1
#             if hit_count != 0 and hit_count % 10 == 0 and not next_button.get_attribute("disabled"):
#                 next_button.click()
#                 tm.sleep(1)

#             downvotes = int(element.find_element(By.XPATH, "./div/div/div/div[2]/div/span").text)
#             hit_count += 1
#             if hit_count != 0 and hit_count % 10 == 0 and not next_button.get_attribute("disabled"):
#                 next_button.click()
#                 tm.sleep(1)

#             similar_perfumes[element.find_element(By.XPATH, "./a").get_attribute("href")] = (upvotes, downvotes)

#         similarly_liked_perfumes = set()
#         similarly_liked_perfumes_elements = new_driver.find_elements(By.CLASS_NAME, 'flickity-slider')[-1].find_elements(By.XPATH, "./div")
#         for element in similarly_liked_perfumes_elements:
#             similarly_liked_perfumes.add(element.find_element(By.XPATH, "./a").get_attribute("href"))

#         frag_info[link] = {}
#         frag_info[link]['name'] = name
#         frag_info[link]['company'] = company
#         frag_info[link]['accords'] = accords
#         frag_info[link]['likability'] = likability
#         frag_info[link]['season'] = season
#         frag_info[link]['time'] = time
#         frag_info[link]['year'] = year
#         frag_info[link]['rating'] = rating
#         frag_info[link]['num_reviews'] = num_reviews
#         frag_info[link]['notes'] = notes
#         frag_info[link]['longevity'] = longevity
#         frag_info[link]['sillage'] = sillage
#         frag_info[link]['gender'] = gender
#         frag_info[link]['price_value'] = price_value
#         frag_info[link]['similar_perfumes'] = similar_perfumes
#         frag_info[link]['similarly_liked_perfumes'] = similarly_liked_perfumes

#         with open("frag_info.pkl", "wb") as f:
#             pickle.dump(frag_info, f)

#         count += 1
#         tm.sleep(6)
#     except URLError as e:
#         print("URLError: " + str(e))
#         print("Retrying...")
#     except TimeoutException as e:
#         print("TimeoutException: " + str(e))
#         print("Retrying...")
#     except ValueError as e:
#         print("ValueError: " + str(e))
#         print("Retrying...")
#     except NoSuchElementException as e:
#         print("NoSuchElementException: " + str(e))
#         if new_driver.current_url == 'https://www.fragrantica.com/':
#             count += 1
#             print("Moving on to next perfume...")
#         else:
#             print("Retrying...")
#     finally:
#         new_driver.execute_script("window.stop();")
#         new_driver.close()
#         new_driver.quit()


In [24]:
# def get_similar_perfumes_and_similarities(perfume):
#     similar_perfumes_and_similarities = {perfume: 1.0}
#     stack = [(perfume, 1.0)]
#     while len(stack) != 0:
#         to_explore, score = stack.pop()
#         to_append = []
#         for similar_perfume in frag_info_copy[to_explore]['similar_perfumes']:
#             upvotes, downvotes = frag_info_copy[to_explore]['similar_perfumes'][similar_perfume]
#             if upvotes >= 7 and score*upvotes*3 >= 2*(upvotes+downvotes):
#                 similar_perfume_similarity = score*upvotes/(upvotes+downvotes)
#                 if similar_perfume not in similar_perfumes_and_similarities or similar_perfume_similarity > similar_perfumes_and_similarities[similar_perfume]:
#                     similar_perfumes_and_similarities[similar_perfume] = similar_perfume_similarity
#                     to_append.append((similar_perfume, similar_perfume_similarity))
#         stack.extend(to_append)
#     return similar_perfumes_and_similarities

# def get_original(similar_perfumes, perfume):
#     min_perfume = perfume
#     min_year = frag_info_copy[perfume]['year']
    
#     for similar_perfume in similar_perfumes:
#         year = frag_info_copy[similar_perfume]['year']
#         if year is not None and (min_year is None or year < min_year or (year == min_year and (frag_info_copy[similar_perfume]['rating'] > frag_info_copy[min_perfume]['rating'] or (frag_info_copy[similar_perfume]['rating'] == frag_info_copy[min_perfume]['rating'] and frag_info_copy[similar_perfume]['num_reviews'] > frag_info_copy[min_perfume]['num_reviews'])))):
#             min_year = year
#             min_perfume = similar_perfume
    
#     return min_perfume

In [14]:
# # add the best version of perfume
# def add_perfume(perfume):
#     print(perfume)
#     if perfume in toget or perfume in forget:
#         return
#     similar_perfumes = list(get_similar_perfumes_and_similarities(perfume).keys())
#     original = get_original(similar_perfumes, perfume)
    
#     similar_perfumes_ratings = [frag_info_copy[similar_perfume]['rating'] for similar_perfume in similar_perfumes]
#     similar_perfumes_ranked = [similar_perfumes[i] for i in np.argsort(similar_perfumes_ratings)[::-1]]
    
#     for similar_perfume in similar_perfumes_ranked:
#         if similar_perfume not in forget and is_male(similar_perfume) and is_well_priced(similar_perfume):
#             if similar_perfume != perfume:
#                 add_perfume(similar_perfume)
#                 add_perfume(perfume)
#                 return
#             elif not is_worthless_knockoff(original, perfume):
#                 toget.add(perfume)
#                 break
#             else:
#                 forget.add(perfume)
#                 continue
#         else:
#             forget.add(similar_perfume)
    
#     for similar_perfume in similar_perfumes:
#         if similar_perfume not in toget:
#             forget.add(similar_perfume)
    
#     return

# def get_gender_points(perfume):
#     return [frag_info_copy[perfume]['gender']['female'],
#                     frag_info_copy[perfume]['gender']['more_female'],
#                     frag_info_copy[perfume]['gender']['unisex'],
#                     frag_info_copy[perfume]['gender']['more_male'],
#                     frag_info_copy[perfume]['gender']['male']]
# def is_male(perfume):
#     gender_points = get_gender_points(perfume)
#     return max(gender_points[3:]) > max(gender_points[:3])

# def get_price_points(perfume):
#     return [frag_info_copy[perfume]['price_value']['way_overpriced'],
#                     frag_info_copy[perfume]['price_value']['overpriced'],
#                     frag_info_copy[perfume]['price_value']['ok'],
#                     frag_info_copy[perfume]['price_value']['good_value'],
#                     frag_info_copy[perfume]['price_value']['great_value']]

# def is_well_priced(perfume):
#     price_points = get_price_points(perfume)
#     return max(price_points[2:]) > max(price_points[:2])

# def get_season(perfume):
#     seasons = ['winter', 'spring', 'summer', 'fall']
#     max_rating = max([frag_info_copy[perfume]['season'][season] for season in seasons])
#     season_return = set()
#     for season in seasons:
#         if frag_info_copy[perfume]['season'][season] == max_rating:
#             season_return.add(season)
#     return season_return

# def get_time(perfume):
#     times = ['day', 'night']
#     max_rating = max([frag_info_copy[perfume]['time'][time] for time in times])
#     time_return = set()
#     for time in times:
#         if frag_info_copy[perfume]['time'][time] == max_rating:
#             time_return.add(time)
#     return time_return
    

# def is_worthless_knockoff(original, perfume):
#     if original == perfume or not is_male(original) or not is_well_priced(original):
#         return False
#     if frag_info_copy[original]['company'] != frag_info_copy[perfume]['company']:
#         if np.argmax(get_price_points(original)) < np.argmax(get_price_points(perfume)):
#             return True
#     return False

# def sort_by_rating(perfumes_list):
#     ratings_numreviews_list = []
#     for perfume in perfumes_list:
#         ratings_numreviews_list.append([frag_info[perfume]['rating'], frag_info[perfume]['num_reviews']])
    

In [15]:
# frag_info_copy = deepcopy(frag_info)

# # delete <100 reviews frags
# for frag in frag_info:
#     if frag_info_copy[frag]['num_reviews'] < 100:
#         del frag_info_copy[frag]

# # make all similar perfume paths two-way
# for frag in frag_info:
#     if frag not in frag_info_copy:
#         continue
#     for similar_frag in frag_info[frag]['similar_perfumes']:
#         if similar_frag not in frag_info_copy:
#             del frag_info_copy[frag]['similar_perfumes'][similar_frag]
#         elif frag not in frag_info_copy[similar_frag]['similar_perfumes']:
#             upvotes, downvotes = frag_info_copy[frag]['similar_perfumes'][similar_frag]
#             frag_info_copy[similar_frag]['similar_perfumes'][frag] = (upvotes, downvotes)

# toget = set()
# forget = set()

# # for perfume in sort_by_ratings(list(frag_info_copy.keys())):
# #     add_perfume(perfume)

In [17]:
# perfumes = list(frag_info_copy.keys())
# ratings = [frag_info_copy[perfume]['rating'] for perfume in perfumes]
# sorted_ratings_idxs = np.argsort(ratings)[::-1]
# sorted_perfumes = [perfumes[i] for i in sorted_ratings_idxs]

In [101]:
# for perfume in sorted_perfumes:
#     if is_male(perfume) and is_well_priced(perfume) and 'winter' in get_season(perfume) and 'day' in get_time(perfume):
#         print(perfume, '\n', frag_info_copy[perfume]['name'], frag_info_copy[perfume]['rating'], '\n\n')

In [102]:
# similar_perfumes = get_similar_perfumes_and_similarities('https://www.fragrantica.com/perfume/Yves-Saint-Laurent/La-Nuit-de-L-Homme-Bleu-Electrique-67997.html')
# for similar_perfume in similar_perfumes:
#     print(similar_perfume, '\n', frag_info_copy[similar_perfume]['name'], frag_info_copy[similar_perfume]['rating'], '\n\n')

In [103]:
# frag_info = {}
# with open("frag_info.pkl", "rb") as f:
#     frag_info = pickle.load(f)

In [12]:
# frag_info_copy = {}
# seasons = ['winter', 'spring', 'summer', 'fall']
# times = ['day', 'night']
# for perfume in frag_info:
#     if frag_info[perfume]['num_reviews'] < 100 or min([frag_info[perfume]['season'][season] for season in seasons]) < 50 or min([frag_info[perfume]['time'][time] for time in times]) < 50:
#         continue
#     frag_info_copy[perfume] = frag_info[perfume]

In [15]:
# perfumes = list(frag_info_copy.keys())
# ratings = [frag_info_copy[perfume]['num_ratings'] for perfume in perfumes]
# sorted_ratings_idxs = np.argsort(ratings)[::-1]
# sorted_perfumes = [perfumes[i] for i in sorted_ratings_idxs]

In [18]:
# perfumes = list(frag_info_copy.keys())
# ratings = [frag_info_copy[perfume]['num_reviews'] for perfume in perfumes]
# sorted_ratings_idxs = np.argsort(ratings)[::-1]
# sorted_perfumes = [perfumes[i] for i in sorted_ratings_idxs]